In [1]:
import pandas as pd
import random
import operator
import math
import sklearn
import numpy as np
from sklearn import metrics

In [17]:
df_full = pd.read_csv(r"C:\Users\chandan sharma\Desktop\research\majorproject\ppr1\code\python\KDD\KDDcup99.csv")#,usecols= range(1,2000))

In [26]:
df_full=df_full.iloc[:400]

In [27]:
from sklearn.preprocessing import LabelEncoder
number=LabelEncoder()
df_full['protocol_type']=number.fit_transform(df_full['protocol_type'].astype('str'))

In [28]:
df_full['service']=number.fit_transform(df_full['service'].astype('str'))

In [29]:
df_full['flag']=number.fit_transform(df_full['flag'].astype('str'))

In [30]:
df_full['label']=number.fit_transform(df_full['label'].astype('str'))

In [31]:
columns = list(df_full.columns)
features = columns[:len(columns)-1]
class_labels = list(df_full[columns[-1]])
df = df_full[features]
num_attr = len(features)
n=len(df)

In [33]:
n

400

In [34]:
# cluster number
k =3
# max number of iterations
max_iter = 10
# Number of data points
n = len(df)
m = 1.80
sm=0
ent=0
pc=0
vpp=0

In [35]:
def vp(membership_mat):
    global vpp
    max1=0
    for i in range(n):
        for j in range(k):
               
            if membership_mat[i][j]>max1:
                max1=membership_mat[i][j];
            else:
                max1=max1;
        vpp+=max1
        #print("________________________",max1)
        #vpp+=f
        nr=math.pow(vpp,2)
    result= nr/n    
    return result

In [36]:
def purity_score(y_true, y_pred):
    # compute contingency matrix (also called confusion matrix)
    contingency_matrix = metrics.cluster.contingency_matrix(y_true, y_pred)
    # return purity
    return np.sum(np.amax(contingency_matrix, axis=0)) / np.sum(contingency_matrix)

In [37]:
def mek(membership_mat):
    global sm
    for j in range(k):
        for i in range(n):
            sm+= membership_mat[i][j]
    #mk=sm/n;
    return sm;        
            

In [38]:
def partitioncoff(membership_mat):
    global pc
    for j in range(k):
        for i in range(n):
            pc+=(membership_mat[i][j]*membership_mat[i][j])
        #print("_____",pc)    
    pc= pc/n;
    return pc   

In [39]:
def entropy(membership_mat):
    global ent
    #count=0;
    for j in range(k):
        for i in range(n):
            g=math.pow(membership_mat[i][j],2)
            ent+= g*(math.log(membership_mat[i][j]))
    res=-(ent)/n
    return res
    #print("_________")

In [40]:
def initializeMembershipMatrix():
    membership_mat = list()
    for i in range(n):
        random_num_list = [random.random() for i in range(k)]
        summation = sum(random_num_list)
        temp_list = [x/summation for x in random_num_list]
        membership_mat.append(temp_list)
    # print(membership_mat)
    return membership_mat


In [41]:
def initializeWeight():
    Weight = [1 for i in range(n)]
    return Weight


In [42]:
def calculateClusterCenter(membership_mat, W):
    #cluster_mem_val = zip(*membership_mat)
    cluster_centers = list()
    for j in range(k):
        x = list(zip(*membership_mat))[j]
        #x = list(cluster_mem_val[j])
        xraised = [e ** m for e in x]
        xraised_mul_W = [a*b for a, b in zip(xraised, W)]
        denominator = sum(xraised_mul_W)
        temp_num = list()
        for i in range(n):
            data_point = list(df.iloc[i])
            prod = [xraised_mul_W[i] *val for val in data_point]
            temp_num.append(prod)
        numerator = map(sum, zip(*temp_num))
        center = [z/denominator for z in numerator]
        cluster_centers.append(center)
    return cluster_centers


In [43]:

def updatemembershipvalue(U, C):
    alpha = float(1/(m-1))
    for i in range(k):
        for j in range(n):
            x = list(df.iloc[j])
            numerator = [(a-b)**2 for a, b in zip(x, C[i])]
            num = sum(numerator)
            dis = [map(operator.sub, x, C[k1]) for k1 in range(k)]
            denominator = [map(lambda x: x**2, dis[j1]) for j1 in range(k)]
            den = [sum(denominator[k1]) for k1 in range(k)]
            res = sum([math.pow(float(num/den[k1]), alpha) for k1 in range(k)])
            U[j][i] = float(1/res)
    return U

In [44]:
def getClusters(membership_mat):
    cluster_labels = list()
    for i in range(n):
        max_val, idx = max((val, idx) for (idx, val) in enumerate(membership_mat[i]))
        cluster_labels.append(idx)
    return cluster_labels

In [45]:
def WFCM():
    W = initializeWeight()
    U = initializeMembershipMatrix()
    center = calculateClusterCenter(U, W)
    i = 0
    while(i <= max_iter):
        U = updatemembershipvalue(U, center)
        center = calculateClusterCenter(U, W)
        cluster_labels = getClusters(U)
        i += 1
   # print(U)
    entro=entropy(U)
    pcvalue=partitioncoff(U) 
    dmk=mek(U)
    findvp=vp(U)
    #print(membership_mat)
    #print("datasize=",n)
    #print("num of cluster=",k)
    #print("entro=",entro) 
    #print("PC=",pcvalue)
    #print("DMK value= ",dmk)
    #print("find vp=",findvp)
    return center, U,cluster_labels

In [46]:
C, U, labels = WFCM()

In [47]:
#print(labels)
print("++++++++======++++")
result=sklearn.metrics.confusion_matrix(labels,class_labels)

++++++++======++++


In [48]:
#print(result)
#labels.sort()
print("silhoutte score::::",metrics.silhouette_score(df,labels,metric='euclidean'))
print("Calinski-harabaz index: ", (metrics.calinski_harabaz_score(df,labels)))
print("purity=",purity_score(class_labels, labels))

silhoutte score:::: 0.7922765104081653
Calinski-harabaz index:  946.4785369309819
purity= 1.0


In [49]:
print("F-measures=",metrics.f1_score(labels,class_labels,average='weighted'))
print("rand index=",metrics.adjusted_rand_score(labels,class_labels))
print("purity score=",purity_score(labels,class_labels))
hello=sklearn.metrics.completeness_score(labels, class_labels)
print("accuracy=",metrics.accuracy_score(labels, class_labels))

F-measures= 0.7952688172043011
rand index= 0.0
purity score= 0.86
accuracy= 0.86


C:\Users\chandan sharma\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
